# KHÁM PHÁ DỮ LIỆU VÀ TIỀN XỬ LÝ

## 1. KHÁM PHÁ DỮ LIỆU

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import warnings

### Đọc dữ liêu tập tin đã crawl về
Đọc từ đường dẫn file `crawl/air_quality/crawldata/air_datas.json` và lưu vào `df`

In [ ]:
path = 'crawl/air_quality/crawldata/air_datas.json'
df = pd.read_json(path)
df.head()

### Kiểm tra dữ liệu có bao nhiêu dòng, bao nhiêu cột
Ta tính số dòng và số cột của DataFrame `df` và lưu vào biến `shape` (tuple). 

In [ ]:
shape = df.shape
shape

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong DataFrame `df` cho biết dữ liệu mức độ ô nhiễm của một thành phố trong một quốc gia trên thế giới 

### Dữ liệu có các dòng bị lặp không?

In [ ]:
id = list(df.index)
num_duplicated_rows = len(id)-len(set(id))
if(num_duplicated_rows == 0):
    print("Dữ liệu không bị lặp")
else: 
    print("Dữ liệu bị lặp " + str(id) + " dòng")

### Mỗi cột có ý nghĩa gì?


In [ ]:
df.columns


### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [ ]:
df.info()

## 2. TIỀN XỬ LÝ

### Nhận xét
- Các cột chủ yếu đều có dạng `numeric`. Tuy nhiên ta có thể thấy được các cột có kiểu dữ kiệu là object (chủ yếu) do đó ta cần phải xử lý đưa kiểu dữ liệu cột về đúng định dạng. 
- Ta đưa về dtype = `np.float64`

In [ ]:
df = df.replace(' - ', np.nan)
warnings.simplefilter("ignore", category = FutureWarning)
df.iloc[:, 3:19] = df.iloc[:, 3:19].astype(np.float64)
df.info()

### Với mỗi cột có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

Với các cột điểm, Ta sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max


Bạn sẽ lưu kết quả vào DataFrame `num_col_info_df`, trong đó: 
- Tên của các cột là:
  - Cột năm: Mức độ ô nhiễm được trong năm 2021
  - Cột tháng: Mức độ ô nhiễm được dự đoán trong tháng của năm 2022
- Tên của các dòng là: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"

Để dễ nhìn, tất cả các giá trị ta đều làm tròn với 1 chữ số thập phân bằng phương thức `.round(1)`.

In [ ]:
def get(col):
    NaNValue = col.isna().sum()
    valu1 = col.dropna()
    x = []
    x.append((NaNValue/len(col))*100)
    x.append(np.percentile(valu1, 0))
    x.append(np.percentile(valu1, 25))
    x.append(np.percentile(valu1, 50))
    x.append(np.percentile(valu1, 75))
    x.append(np.percentile(valu1, 100))
    return np.array(x).round(1)

index_info=["missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"]
dict_info = {}
for i in df.columns:
    if df[i].dtype in [np.float32, np.float64]:
        dict_info[i] = get(df[i])   
num_col_info_df =pd.DataFrame(dict_info,index_info)
num_col_info_df

### Tiền xử lý các cột của dữ liệu
Trước khi ghi dữ liệu tiền xử lý ra file csv, ta thấy cột city có thể tách làm 2 cột riêng là cột city và cột country để dễ dàng thực hiện cho các thao tác sau này.

In [ ]:
split_city_df = df['City'].str.split(', ')
df['City'] = split_city_df.str[0]
df.insert(2, 'Country', split_city_df.str[1], True)
df.head()

Nhìn chung do cách crawl data về không phù hợp nên các dòng dữ liệu bị sắp xếp lộn xộn, do đó ta sẽ sử dụng sort_values để sắp lại theo thứ tự tăng dần cột Rank

In [ ]:
df.sort_values(by = ['Rank'], inplace = True)

### Sau đó ta hãy ghi df ra file `airdata6years.csv` vào folder data

In [ ]:
df.to_csv('./../data/airdata6years.csv')